In [1]:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./train_new.csv', encoding='utf-8')
test = pd.read_csv('./test_new.csv', encoding='utf-8')

In [3]:
train.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,1814,190,191,2404,...,069044,069050,709031,899005,899003,30003,159063,3816,3818,300003
0,002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,32.0,55.0,215.0,166.5,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,92dd479df5e30ab6a0a1cf85ac53efc3,141.0,97.0,2.64,1.36,4.75,189.0,71.0,376.0,181.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,6bb59d517c4c70f8f50844d24fbd0355,120.0,80.0,1.37,1.25,2.66,281.0,78.0,436.0,173.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,0ebb42adae512906f7e1135da734ea63,100.0,70.0,1.27,2.21,1.73,26.0,65.9,205.0,161.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,ebe7811e919109c42c092abbd98b4ca6,110.0,80.0,0.80,1.87,2.21,24.0,46.0,236.0,158.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [4]:
test.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,1814,190,191,2404,...,069044,069050,709031,899005,899003,30003,159063,3816,3818,300003
0,c643a744e2e94f3ff354d920958bd37b,153.729950,80.836021,2.495490,1.626395,3.292172,16.00,43.00,321.00,149.0,...,-999.0,-999.0,-999.0,101.8,8.68,-999.0,-999.0,-999.0,-999.0,-999.0
1,a81c1fc0cfa8c0fbc450d2ffeae14b9f,141.165848,87.159599,2.734281,1.337232,2.955503,20.00,75.60,583.00,172.0,...,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0
2,2af0da8f6f0b8d642f88b3a8194408b6,119.323578,74.147018,1.363414,1.416480,2.588616,16.00,62.00,379.00,154.0,...,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0
3,2bd361a4c5b9b0d5aab62c101daa4ee6,125.918457,78.032928,2.631400,1.202706,3.069483,74.83,63.98,429.08,178.5,...,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0
4,16107e514ed0f51c4e39b1a0fa5a04f9,136.977875,80.865005,1.126695,1.408028,2.337502,21.36,74.07,255.60,168.5,...,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0


In [5]:
target_feat = ['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']

In [6]:
len(train)

38192

In [11]:
for feat in target_feat:
    train[feat].replace('未查', np.nan, inplace=True)
    train[feat].replace('弃查', np.nan, inplace=True)
    train[feat].replace(to_replace='[^0-9.]+', value='', regex=True, inplace=True)
    train[feat].replace(to_replace='2.2.8', value='2.28', inplace=True)
    train[feat] = train[feat].astype(float)
    # 只是针对target的缺失值填充平均值
    train[feat].fillna(train[feat].mean)
    #train.dropna(subset=[feat], inplace=True)

In [12]:
len(train)

38192

In [13]:
features = [feat for feat in train.columns if feat not in target_feat + ['vid']]

In [14]:
train_data = train[features]
test_data = test[features]
X_train, X_test, y_train, y_test = train_test_split(train_data, train[target_feat],
                                                   test_size = 0.15,
                                                   random_state=100)

In [15]:
X_test_pred = pd.DataFrame()

In [16]:
# 定义评价函数
def evaluation(pred, target):
    target = target.get_label()
    bias = np.ones((1, len(pred)))
    log_pred = np.log(pred + bias)
    log_target = np.log(target + bias)
    distance = log_pred - log_target
    distance_sq = np.square(distance)
    loss = np.sum(distance_sq) / float(len(pred))
    return 'loss_value', float(loss)

In [17]:
# 收缩压
sp_model = xgb.XGBRegressor(n_estimators = 1000, max_depth = 5, learning_rate = 0.1)
# sp_model = xgb.XGBRegressor(n_estimators=500, max_depth=5, learning_rate=0.1)
print(sp_model)
# sp_model = xgb.XGBRegressor(n_estimators=1000, 
#                         max_depth=9, 
#                         learning_rate=0.03, 
#                         subsample=0.8, 
#                         silent=True, 
#                         seed=1,
#                         objective='reg:linear', 
#                         reg_alpha=1, 
#                         reg_lambda=1, 
#                         gamma=0, 
#                         missing=None, 
#                         colsample_bytree=0.3,
#                         scale_pos_weight=1, 
#                         min_child_weight=1, 
#                         max_delta_step=0, 
#                         base_socre=0.5)

sp_model.fit(X_train, y_train['收缩压'], eval_metric=evaluation, verbose=True, eval_set=[(X_test, y_test['收缩压'])], 
             early_stopping_rounds=100)
X_test_pred['收缩压'] = sp_model.predict(X_test)

# sp_model.fit(train_data, train['收缩压'])
test['收缩压'] = sp_model.predict(test_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
[0]	validation_0-rmse:114.685	validation_0-loss_value:4.82409
Multiple eval metrics have been passed: 'validation_0-loss_value' will be used for early stopping.

Will train until validation_0-loss_value hasn't improved in 100 rounds.
[1]	validation_0-rmse:103.507	validation_0-loss_value:2.58546
[2]	validation_0-rmse:93.4756	validation_0-loss_value:1.61845
[3]	validation_0-rmse:84.4907	validation_0-loss_value:1.09148
[4]	validation_0-rmse:76.438	validation_0-loss_value:0.769643
[5]	validation_0-rmse:69.2185	validation_0-loss_value:0.559227
[6]	validation_0-rmse:62.7745	validation_0-loss_value:0.415932
[7]	valid

[120]	validation_0-rmse:16.7171	validation_0-loss_value:0.016212
[121]	validation_0-rmse:16.7166	validation_0-loss_value:0.016212
[122]	validation_0-rmse:16.716	validation_0-loss_value:0.016212
[123]	validation_0-rmse:16.7157	validation_0-loss_value:0.016211
[124]	validation_0-rmse:16.7156	validation_0-loss_value:0.016211
[125]	validation_0-rmse:16.7173	validation_0-loss_value:0.016214
[126]	validation_0-rmse:16.7147	validation_0-loss_value:0.016209
[127]	validation_0-rmse:16.7141	validation_0-loss_value:0.016208
[128]	validation_0-rmse:16.7102	validation_0-loss_value:0.016201
[129]	validation_0-rmse:16.7068	validation_0-loss_value:0.016195
[130]	validation_0-rmse:16.7066	validation_0-loss_value:0.016195
[131]	validation_0-rmse:16.7025	validation_0-loss_value:0.016188
[132]	validation_0-rmse:16.7022	validation_0-loss_value:0.016187
[133]	validation_0-rmse:16.7003	validation_0-loss_value:0.016184
[134]	validation_0-rmse:16.6972	validation_0-loss_value:0.01618
[135]	validation_0-rmse:16.

[247]	validation_0-rmse:16.6512	validation_0-loss_value:0.016101
[248]	validation_0-rmse:16.6521	validation_0-loss_value:0.016102
[249]	validation_0-rmse:16.6514	validation_0-loss_value:0.016101
[250]	validation_0-rmse:16.6525	validation_0-loss_value:0.016104
[251]	validation_0-rmse:16.6551	validation_0-loss_value:0.01611
[252]	validation_0-rmse:16.6531	validation_0-loss_value:0.016106
[253]	validation_0-rmse:16.6528	validation_0-loss_value:0.016104
[254]	validation_0-rmse:16.6523	validation_0-loss_value:0.016104
[255]	validation_0-rmse:16.6528	validation_0-loss_value:0.016104
[256]	validation_0-rmse:16.6534	validation_0-loss_value:0.016105
[257]	validation_0-rmse:16.654	validation_0-loss_value:0.016107
[258]	validation_0-rmse:16.6533	validation_0-loss_value:0.016106
[259]	validation_0-rmse:16.6539	validation_0-loss_value:0.016108
[260]	validation_0-rmse:16.6532	validation_0-loss_value:0.016107
[261]	validation_0-rmse:16.6535	validation_0-loss_value:0.016107
[262]	validation_0-rmse:16.

In [18]:
# 舒张压
dp_model = xgb.XGBRegressor(n_estimators = 1000, max_depth = 5, learning_rate = 0.1)
print(dp_model)

# dp_model = xgb.XGBRegressor(n_estimators=1000, 
#                        max_depth=9, 
#                        learning_rate=0.03, 
#                        subsample=0.8, 
#                        silent=True, 
#                        seed=1,
#                        objective='reg:linear', 
#                        reg_alpha=1, 
#                        reg_lambda=1,
#                        gamma=0, 
#                        missing=None, 
#                        colsample_bytree=0.3,
#                        scale_pos_weight=1, 
#                        min_child_weight=1, 
#                        max_delta_step=0, 
#                        base_socre=0.5
#                        )

dp_model.fit(X_train, y_train['舒张压'], eval_metric=evaluation, verbose=True, eval_set=[(X_test, y_test['舒张压'])], 
             early_stopping_rounds=100)
X_test_pred['舒张压'] = dp_model.predict(X_test)

# dp_model.fit(train_data, train['舒张压'])
test['舒张压'] = dp_model.predict(test_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
[0]	validation_0-rmse:69.8978	validation_0-loss_value:4.5627
Multiple eval metrics have been passed: 'validation_0-loss_value' will be used for early stopping.

Will train until validation_0-loss_value hasn't improved in 100 rounds.
[1]	validation_0-rmse:63.1015	validation_0-loss_value:2.48645
[2]	validation_0-rmse:57.0042	validation_0-loss_value:1.56719
[3]	validation_0-rmse:51.5394	validation_0-loss_value:1.06066
[4]	validation_0-rmse:46.6441	validation_0-loss_value:0.749748
[5]	validation_0-rmse:42.2631	validation_0-loss_value:0.546018
[6]	validation_0-rmse:38.3451	validation_0-loss_value:0.406663
[7]	valid

[120]	validation_0-rmse:10.8946	validation_0-loss_value:0.019244
[121]	validation_0-rmse:10.8949	validation_0-loss_value:0.019244
[122]	validation_0-rmse:10.8919	validation_0-loss_value:0.019234
[123]	validation_0-rmse:10.8902	validation_0-loss_value:0.019226
[124]	validation_0-rmse:10.8899	validation_0-loss_value:0.019224
[125]	validation_0-rmse:10.8892	validation_0-loss_value:0.019222
[126]	validation_0-rmse:10.8881	validation_0-loss_value:0.019219
[127]	validation_0-rmse:10.8844	validation_0-loss_value:0.019206
[128]	validation_0-rmse:10.8836	validation_0-loss_value:0.019203
[129]	validation_0-rmse:10.8852	validation_0-loss_value:0.019209
[130]	validation_0-rmse:10.8851	validation_0-loss_value:0.019209
[131]	validation_0-rmse:10.8862	validation_0-loss_value:0.019213
[132]	validation_0-rmse:10.8862	validation_0-loss_value:0.019213
[133]	validation_0-rmse:10.8861	validation_0-loss_value:0.019212
[134]	validation_0-rmse:10.8857	validation_0-loss_value:0.019209
[135]	validation_0-rmse:1

[247]	validation_0-rmse:10.8835	validation_0-loss_value:0.019174
[248]	validation_0-rmse:10.8834	validation_0-loss_value:0.019174
[249]	validation_0-rmse:10.8822	validation_0-loss_value:0.019172
[250]	validation_0-rmse:10.8828	validation_0-loss_value:0.019174
[251]	validation_0-rmse:10.8822	validation_0-loss_value:0.019172
[252]	validation_0-rmse:10.8809	validation_0-loss_value:0.019168
[253]	validation_0-rmse:10.8828	validation_0-loss_value:0.019174
[254]	validation_0-rmse:10.8838	validation_0-loss_value:0.019176
[255]	validation_0-rmse:10.8824	validation_0-loss_value:0.019172
[256]	validation_0-rmse:10.8829	validation_0-loss_value:0.019173
[257]	validation_0-rmse:10.8812	validation_0-loss_value:0.019168
[258]	validation_0-rmse:10.883	validation_0-loss_value:0.019174
[259]	validation_0-rmse:10.8834	validation_0-loss_value:0.019175
[260]	validation_0-rmse:10.8835	validation_0-loss_value:0.019176
[261]	validation_0-rmse:10.8837	validation_0-loss_value:0.019176
[262]	validation_0-rmse:10

In [19]:
# 血清甘油三酯
tg_model = xgb.XGBRegressor(n_estimators=1000, 
                       max_depth=5, 
                       learning_rate=0.1)
# tg_model = xgb.XGBRegressor(n_estimators=1000, 
#                        max_depth=9, 
#                        learning_rate=0.03, 
#                        subsample=0.8, 
#                        silent=True, 
#                        seed=1,
#                        objective='reg:linear', 
#                        reg_alpha=0.5, 
#                        reg_lambda=1, 
#                        gamma=0, 
#                        missing=None, 
#                        colsample_bytree=0.3,
#                        scale_pos_weight=1, 
#                        min_child_weight=1, 
#                        max_delta_step=0, 
#                        base_socre=0.5)

tg_model.fit(X_train, y_train['血清甘油三酯'], eval_metric=evaluation, verbose=True, eval_set=[(X_test, y_test['血清甘油三酯'])], 
             early_stopping_rounds=100)
X_test_pred['血清甘油三酯'] = tg_model.predict(X_test)

# tg_model.fit(train_data, train['血清甘油三酯'])
test['血清甘油三酯'] = tg_model.predict(test_data)

[0]	validation_0-rmse:1.62484	validation_0-loss_value:0.286474
Multiple eval metrics have been passed: 'validation_0-loss_value' will be used for early stopping.

Will train until validation_0-loss_value hasn't improved in 100 rounds.
[1]	validation_0-rmse:1.54513	validation_0-loss_value:0.233329
[2]	validation_0-rmse:1.47777	validation_0-loss_value:0.194977
[3]	validation_0-rmse:1.42105	validation_0-loss_value:0.167168
[4]	validation_0-rmse:1.37231	validation_0-loss_value:0.146222
[5]	validation_0-rmse:1.32627	validation_0-loss_value:0.130355
[6]	validation_0-rmse:1.29256	validation_0-loss_value:0.119028
[7]	validation_0-rmse:1.26031	validation_0-loss_value:0.109989
[8]	validation_0-rmse:1.23672	validation_0-loss_value:0.103474
[9]	validation_0-rmse:1.21692	validation_0-loss_value:0.098517
[10]	validation_0-rmse:1.20021	validation_0-loss_value:0.09485
[11]	validation_0-rmse:1.18566	validation_0-loss_value:0.09193
[12]	validation_0-rmse:1.17473	validation_0-loss_value:0.089991
[13]	val

[126]	validation_0-rmse:1.09221	validation_0-loss_value:0.082323
[127]	validation_0-rmse:1.09243	validation_0-loss_value:0.082358
[128]	validation_0-rmse:1.09271	validation_0-loss_value:0.08239
[129]	validation_0-rmse:1.09267	validation_0-loss_value:0.082382
[130]	validation_0-rmse:1.09242	validation_0-loss_value:0.082341
[131]	validation_0-rmse:1.09259	validation_0-loss_value:0.082368
[132]	validation_0-rmse:1.0922	validation_0-loss_value:0.082307
[133]	validation_0-rmse:1.09243	validation_0-loss_value:0.082316
[134]	validation_0-rmse:1.09238	validation_0-loss_value:0.08233
[135]	validation_0-rmse:1.09243	validation_0-loss_value:0.082318
[136]	validation_0-rmse:1.09235	validation_0-loss_value:0.0823
[137]	validation_0-rmse:1.09282	validation_0-loss_value:0.082395
[138]	validation_0-rmse:1.09302	validation_0-loss_value:0.082392
[139]	validation_0-rmse:1.09284	validation_0-loss_value:0.082366
[140]	validation_0-rmse:1.09279	validation_0-loss_value:0.082343
[141]	validation_0-rmse:1.0929

In [20]:
# 血清高密度脂蛋白
hdl_model = xgb.XGBRegressor(n_estimators=1000, 
                       max_depth=5, 
                       learning_rate=0.1)
# hdl_model = xgb.XGBRegressor(n_estimators=1000, 
#                        max_depth=9, 
#                        learning_rate=0.03, 
#                        subsample=0.8, 
#                        silent=True, 
#                        seed=1,
#                        objective='reg:linear', 
#                        reg_alpha=0.5, 
#                        reg_lambda=1, 
#                        gamma=0, 
#                        missing=None, 
#                        colsample_bytree=0.3,
#                        scale_pos_weight=1, 
#                        min_child_weight=1, 
#                        max_delta_step=0, 
#                        base_socre=0.5)

hdl_model.fit(X_train, y_train['血清高密度脂蛋白'], eval_metric=evaluation, verbose=True, eval_set=[(X_test, y_test['血清高密度脂蛋白'])], 
              early_stopping_rounds=100)
X_test_pred['血清高密度脂蛋白'] = hdl_model.predict(X_test)

# hdl_model.fit(train_data, train['血清高密度脂蛋白'])
test['血清高密度脂蛋白'] = hdl_model.predict(test_data)

[0]	validation_0-rmse:0.880353	validation_0-loss_value:0.180744
Multiple eval metrics have been passed: 'validation_0-loss_value' will be used for early stopping.

Will train until validation_0-loss_value hasn't improved in 100 rounds.
[1]	validation_0-rmse:0.804294	validation_0-loss_value:0.14251
[2]	validation_0-rmse:0.737041	validation_0-loss_value:0.113759
[3]	validation_0-rmse:0.677429	validation_0-loss_value:0.091818
[4]	validation_0-rmse:0.624909	validation_0-loss_value:0.074988
[5]	validation_0-rmse:0.578764	validation_0-loss_value:0.061998
[6]	validation_0-rmse:0.538467	validation_0-loss_value:0.051926
[7]	validation_0-rmse:0.503474	validation_0-loss_value:0.044099
[8]	validation_0-rmse:0.473148	validation_0-loss_value:0.037969
[9]	validation_0-rmse:0.446998	validation_0-loss_value:0.033158
[10]	validation_0-rmse:0.424631	validation_0-loss_value:0.029384
[11]	validation_0-rmse:0.405142	validation_0-loss_value:0.026352
[12]	validation_0-rmse:0.388649	validation_0-loss_value:0.0

[124]	validation_0-rmse:0.288155	validation_0-loss_value:0.013428
[125]	validation_0-rmse:0.288113	validation_0-loss_value:0.013424
[126]	validation_0-rmse:0.287972	validation_0-loss_value:0.013411
[127]	validation_0-rmse:0.287939	validation_0-loss_value:0.013411
[128]	validation_0-rmse:0.287946	validation_0-loss_value:0.013412
[129]	validation_0-rmse:0.287976	validation_0-loss_value:0.013415
[130]	validation_0-rmse:0.287907	validation_0-loss_value:0.013409
[131]	validation_0-rmse:0.287857	validation_0-loss_value:0.013404
[132]	validation_0-rmse:0.287829	validation_0-loss_value:0.013401
[133]	validation_0-rmse:0.287748	validation_0-loss_value:0.013395
[134]	validation_0-rmse:0.287693	validation_0-loss_value:0.01339
[135]	validation_0-rmse:0.287653	validation_0-loss_value:0.013386
[136]	validation_0-rmse:0.287621	validation_0-loss_value:0.013383
[137]	validation_0-rmse:0.28757	validation_0-loss_value:0.013379
[138]	validation_0-rmse:0.28741	validation_0-loss_value:0.013365
[139]	validat

[249]	validation_0-rmse:0.283695	validation_0-loss_value:0.013035
[250]	validation_0-rmse:0.283663	validation_0-loss_value:0.013032
[251]	validation_0-rmse:0.283656	validation_0-loss_value:0.013031
[252]	validation_0-rmse:0.283669	validation_0-loss_value:0.013032
[253]	validation_0-rmse:0.283635	validation_0-loss_value:0.013028
[254]	validation_0-rmse:0.283612	validation_0-loss_value:0.013026
[255]	validation_0-rmse:0.283606	validation_0-loss_value:0.013026
[256]	validation_0-rmse:0.283582	validation_0-loss_value:0.013023
[257]	validation_0-rmse:0.283561	validation_0-loss_value:0.013022
[258]	validation_0-rmse:0.283572	validation_0-loss_value:0.013023
[259]	validation_0-rmse:0.28355	validation_0-loss_value:0.013021
[260]	validation_0-rmse:0.283469	validation_0-loss_value:0.013014
[261]	validation_0-rmse:0.283433	validation_0-loss_value:0.01301
[262]	validation_0-rmse:0.283415	validation_0-loss_value:0.013009
[263]	validation_0-rmse:0.283424	validation_0-loss_value:0.013009
[264]	valida

[374]	validation_0-rmse:0.281891	validation_0-loss_value:0.012878
[375]	validation_0-rmse:0.281895	validation_0-loss_value:0.012877
[376]	validation_0-rmse:0.2819	validation_0-loss_value:0.012878
[377]	validation_0-rmse:0.281899	validation_0-loss_value:0.012878
[378]	validation_0-rmse:0.281922	validation_0-loss_value:0.01288
[379]	validation_0-rmse:0.281923	validation_0-loss_value:0.01288
[380]	validation_0-rmse:0.281898	validation_0-loss_value:0.012878
[381]	validation_0-rmse:0.281929	validation_0-loss_value:0.012881
[382]	validation_0-rmse:0.281905	validation_0-loss_value:0.012879
[383]	validation_0-rmse:0.281906	validation_0-loss_value:0.012879
[384]	validation_0-rmse:0.281915	validation_0-loss_value:0.012879
[385]	validation_0-rmse:0.281907	validation_0-loss_value:0.012879
[386]	validation_0-rmse:0.281888	validation_0-loss_value:0.012877
[387]	validation_0-rmse:0.281839	validation_0-loss_value:0.012872
[388]	validation_0-rmse:0.281821	validation_0-loss_value:0.012871
[389]	validati

[499]	validation_0-rmse:0.281198	validation_0-loss_value:0.012818
[500]	validation_0-rmse:0.281169	validation_0-loss_value:0.012815
[501]	validation_0-rmse:0.281204	validation_0-loss_value:0.012818
[502]	validation_0-rmse:0.281213	validation_0-loss_value:0.012819
[503]	validation_0-rmse:0.281214	validation_0-loss_value:0.012819
[504]	validation_0-rmse:0.281212	validation_0-loss_value:0.012819
[505]	validation_0-rmse:0.281228	validation_0-loss_value:0.01282
[506]	validation_0-rmse:0.281226	validation_0-loss_value:0.01282
[507]	validation_0-rmse:0.281239	validation_0-loss_value:0.012821
[508]	validation_0-rmse:0.281244	validation_0-loss_value:0.012822
[509]	validation_0-rmse:0.28125	validation_0-loss_value:0.012822
[510]	validation_0-rmse:0.281258	validation_0-loss_value:0.012823
[511]	validation_0-rmse:0.281266	validation_0-loss_value:0.012824
[512]	validation_0-rmse:0.281262	validation_0-loss_value:0.012824
[513]	validation_0-rmse:0.281243	validation_0-loss_value:0.012822
[514]	validat

[624]	validation_0-rmse:0.280963	validation_0-loss_value:0.012805
[625]	validation_0-rmse:0.28096	validation_0-loss_value:0.012804
[626]	validation_0-rmse:0.280993	validation_0-loss_value:0.012807
[627]	validation_0-rmse:0.280979	validation_0-loss_value:0.012806
[628]	validation_0-rmse:0.281	validation_0-loss_value:0.012808
[629]	validation_0-rmse:0.281001	validation_0-loss_value:0.012808
[630]	validation_0-rmse:0.281022	validation_0-loss_value:0.01281
[631]	validation_0-rmse:0.281029	validation_0-loss_value:0.012811
[632]	validation_0-rmse:0.281027	validation_0-loss_value:0.012811
[633]	validation_0-rmse:0.28104	validation_0-loss_value:0.012813
[634]	validation_0-rmse:0.281049	validation_0-loss_value:0.012813
[635]	validation_0-rmse:0.281042	validation_0-loss_value:0.012812
[636]	validation_0-rmse:0.281067	validation_0-loss_value:0.012816
[637]	validation_0-rmse:0.281058	validation_0-loss_value:0.012815
[638]	validation_0-rmse:0.281081	validation_0-loss_value:0.012817
[639]	validation

In [21]:
# 血清低密度脂蛋白
ldl_model = xgb.XGBRegressor(n_estimators=1000, 
                       max_depth=5, 
                       learning_rate=0.1)
print(ldl_model)
# ldl_model = xgb.XGBRegressor(n_estimators=1000, 
#                        max_depth=9, 
#                        learning_rate=0.03, 
#                        subsample=0.8, 
#                        silent=True, 
#                        seed=1,
#                        objective='reg:linear', 
#                        reg_alpha=0.5, 
#                        reg_lambda=1, 
#                        gamma=0, 
#                        missing=None, 
#                        colsample_bytree=0.3,
#                        scale_pos_weight=1, 
#                        min_child_weight=1, 
#                        max_delta_step=0, 
#                        base_socre=0.5)

ldl_model.fit(X_train, y_train['血清低密度脂蛋白'], eval_metric=evaluation, verbose=True, eval_set=[(X_test, y_test['血清低密度脂蛋白'])], 
              early_stopping_rounds=100)
X_test_pred['血清低密度脂蛋白'] = ldl_model.predict(X_test)

# ldl_model.fit(train_data, train['血清低密度脂蛋白'])
test['血清低密度脂蛋白'] = ldl_model.predict(test_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
[0]	validation_0-rmse:2.21515	validation_0-loss_value:0.620546
Multiple eval metrics have been passed: 'validation_0-loss_value' will be used for early stopping.

Will train until validation_0-loss_value hasn't improved in 100 rounds.
[1]	validation_0-rmse:2.02513	validation_0-loss_value:0.463253
[2]	validation_0-rmse:1.85654	validation_0-loss_value:0.353604
[3]	validation_0-rmse:1.70724	validation_0-loss_value:0.275229
[4]	validation_0-rmse:1.57607	validation_0-loss_value:0.218302
[5]	validation_0-rmse:1.461	validation_0-loss_value:0.176299
[6]	validation_0-rmse:1.36045	validation_0-loss_value:0.14483
[7]	val

[118]	validation_0-rmse:0.734021	validation_0-loss_value:0.037778
[119]	validation_0-rmse:0.733835	validation_0-loss_value:0.037759
[120]	validation_0-rmse:0.733787	validation_0-loss_value:0.037755
[121]	validation_0-rmse:0.733586	validation_0-loss_value:0.037727
[122]	validation_0-rmse:0.733322	validation_0-loss_value:0.037702
[123]	validation_0-rmse:0.733277	validation_0-loss_value:0.037696
[124]	validation_0-rmse:0.732879	validation_0-loss_value:0.037655
[125]	validation_0-rmse:0.732849	validation_0-loss_value:0.037654
[126]	validation_0-rmse:0.732787	validation_0-loss_value:0.037646
[127]	validation_0-rmse:0.732467	validation_0-loss_value:0.037614
[128]	validation_0-rmse:0.732249	validation_0-loss_value:0.037591
[129]	validation_0-rmse:0.732216	validation_0-loss_value:0.037587
[130]	validation_0-rmse:0.732225	validation_0-loss_value:0.037587
[131]	validation_0-rmse:0.732135	validation_0-loss_value:0.037577
[132]	validation_0-rmse:0.732118	validation_0-loss_value:0.037574
[133]	vali

[243]	validation_0-rmse:0.724754	validation_0-loss_value:0.036823
[244]	validation_0-rmse:0.724736	validation_0-loss_value:0.036822
[245]	validation_0-rmse:0.724731	validation_0-loss_value:0.036819
[246]	validation_0-rmse:0.724679	validation_0-loss_value:0.036815
[247]	validation_0-rmse:0.724624	validation_0-loss_value:0.03681
[248]	validation_0-rmse:0.724667	validation_0-loss_value:0.036817
[249]	validation_0-rmse:0.724626	validation_0-loss_value:0.03681
[250]	validation_0-rmse:0.724668	validation_0-loss_value:0.036811
[251]	validation_0-rmse:0.724586	validation_0-loss_value:0.036803
[252]	validation_0-rmse:0.72442	validation_0-loss_value:0.036784
[253]	validation_0-rmse:0.724499	validation_0-loss_value:0.036792
[254]	validation_0-rmse:0.72438	validation_0-loss_value:0.036776
[255]	validation_0-rmse:0.724366	validation_0-loss_value:0.036774
[256]	validation_0-rmse:0.724423	validation_0-loss_value:0.036779
[257]	validation_0-rmse:0.724403	validation_0-loss_value:0.03678
[258]	validatio

[368]	validation_0-rmse:0.722468	validation_0-loss_value:0.036568
[369]	validation_0-rmse:0.722499	validation_0-loss_value:0.036571
[370]	validation_0-rmse:0.722465	validation_0-loss_value:0.036568
[371]	validation_0-rmse:0.722484	validation_0-loss_value:0.03657
[372]	validation_0-rmse:0.722438	validation_0-loss_value:0.036567
[373]	validation_0-rmse:0.722337	validation_0-loss_value:0.03656
[374]	validation_0-rmse:0.722253	validation_0-loss_value:0.036552
[375]	validation_0-rmse:0.72199	validation_0-loss_value:0.036522
[376]	validation_0-rmse:0.722002	validation_0-loss_value:0.036521
[377]	validation_0-rmse:0.721993	validation_0-loss_value:0.036523
[378]	validation_0-rmse:0.721868	validation_0-loss_value:0.036508
[379]	validation_0-rmse:0.721777	validation_0-loss_value:0.0365
[380]	validation_0-rmse:0.721812	validation_0-loss_value:0.036502
[381]	validation_0-rmse:0.721782	validation_0-loss_value:0.0365
[382]	validation_0-rmse:0.721825	validation_0-loss_value:0.036503
[383]	validation_

[493]	validation_0-rmse:0.72109	validation_0-loss_value:0.036414
[494]	validation_0-rmse:0.720997	validation_0-loss_value:0.036405
[495]	validation_0-rmse:0.720978	validation_0-loss_value:0.036403
[496]	validation_0-rmse:0.721006	validation_0-loss_value:0.036404
[497]	validation_0-rmse:0.721034	validation_0-loss_value:0.036407
[498]	validation_0-rmse:0.72108	validation_0-loss_value:0.036414
[499]	validation_0-rmse:0.721079	validation_0-loss_value:0.036414
[500]	validation_0-rmse:0.721109	validation_0-loss_value:0.036417
[501]	validation_0-rmse:0.721099	validation_0-loss_value:0.036416
[502]	validation_0-rmse:0.721119	validation_0-loss_value:0.036417
[503]	validation_0-rmse:0.721087	validation_0-loss_value:0.036413
[504]	validation_0-rmse:0.721065	validation_0-loss_value:0.03641
[505]	validation_0-rmse:0.721082	validation_0-loss_value:0.036412
[506]	validation_0-rmse:0.721061	validation_0-loss_value:0.036411
[507]	validation_0-rmse:0.72109	validation_0-loss_value:0.036414
[508]	validati

[618]	validation_0-rmse:0.720232	validation_0-loss_value:0.036315
[619]	validation_0-rmse:0.720266	validation_0-loss_value:0.036317
[620]	validation_0-rmse:0.720305	validation_0-loss_value:0.036321
[621]	validation_0-rmse:0.720236	validation_0-loss_value:0.036316
[622]	validation_0-rmse:0.720298	validation_0-loss_value:0.036323
[623]	validation_0-rmse:0.720344	validation_0-loss_value:0.036326
[624]	validation_0-rmse:0.720349	validation_0-loss_value:0.036328
[625]	validation_0-rmse:0.720348	validation_0-loss_value:0.036329
[626]	validation_0-rmse:0.720409	validation_0-loss_value:0.036333
[627]	validation_0-rmse:0.720468	validation_0-loss_value:0.036339
[628]	validation_0-rmse:0.720432	validation_0-loss_value:0.036337
[629]	validation_0-rmse:0.72044	validation_0-loss_value:0.036338
[630]	validation_0-rmse:0.720511	validation_0-loss_value:0.036347
[631]	validation_0-rmse:0.72051	validation_0-loss_value:0.036346
[632]	validation_0-rmse:0.720502	validation_0-loss_value:0.036345
[633]	valida

[743]	validation_0-rmse:0.720348	validation_0-loss_value:0.03631
[744]	validation_0-rmse:0.720305	validation_0-loss_value:0.036306
[745]	validation_0-rmse:0.720329	validation_0-loss_value:0.036309
[746]	validation_0-rmse:0.720322	validation_0-loss_value:0.036307
[747]	validation_0-rmse:0.720318	validation_0-loss_value:0.036306
[748]	validation_0-rmse:0.720312	validation_0-loss_value:0.036305
[749]	validation_0-rmse:0.720341	validation_0-loss_value:0.036307
[750]	validation_0-rmse:0.720338	validation_0-loss_value:0.036308
[751]	validation_0-rmse:0.720396	validation_0-loss_value:0.036313
[752]	validation_0-rmse:0.720402	validation_0-loss_value:0.036311
[753]	validation_0-rmse:0.720435	validation_0-loss_value:0.036313
[754]	validation_0-rmse:0.720437	validation_0-loss_value:0.036314
[755]	validation_0-rmse:0.720374	validation_0-loss_value:0.036311
[756]	validation_0-rmse:0.720328	validation_0-loss_value:0.036307
[757]	validation_0-rmse:0.72042	validation_0-loss_value:0.036311
[758]	valida